### Breakthrough Curve Analysis

The following plot shows the predicted breakthrough behavior from PHREEQC simulation:

In [ ]:
# Load breakthrough data if available
breakthrough_curve_path = locals().get('breakthrough_curve_path', None)

if breakthrough_data and 'bv' in breakthrough_data:
    # Use breakthrough_data from simulation (new format)
    bv = breakthrough_data['bv']
    hardness = breakthrough_data.get('hardness_mg_l', [])
    ca = breakthrough_data.get('ca_mg_l', [])
    mg = breakthrough_data.get('mg_mg_l', [])
    na = breakthrough_data.get('na_mg_l', [])
    
    # Find service endpoint
    target_hardness = design.get('targets', {}).get('hardness_mg_l_caco3', 5.0)
    breakthrough_bv = performance.get('service_bv_to_target', 0)
    
    print(f"Breakthrough at {breakthrough_bv:.1f} BV for {target_hardness:.1f} mg/L target")
elif breakthrough_data and 'bed_volumes' in breakthrough_data:
    # Legacy format compatibility
    bv = breakthrough_data['bed_volumes']
    hardness = breakthrough_data.get('hardness_mg_l', [])
    ca = breakthrough_data.get('ca_mg_l', [])
    mg = breakthrough_data.get('mg_mg_l', [])
    na = breakthrough_data.get('na_mg_l', [])
    
    target_hardness = breakthrough_data.get('target_hardness', 5.0)
    breakthrough_bv = breakthrough_data.get('breakthrough_bv', performance.get('service_bv_to_target', 0))
    
    print(f"Breakthrough at {breakthrough_bv:.1f} BV for {target_hardness:.1f} mg/L target")
elif breakthrough_curve_path:
    # Try to load from CSV file
    try:
        df_breakthrough = pd.read_csv(breakthrough_curve_path)
        bv = df_breakthrough['bed_volumes'].values
        hardness = df_breakthrough['hardness_mg_l'].values
        target_hardness = 5.0  # Default target
        breakthrough_bv = performance.get('service_bv_to_target', 0)
        print(f"Loaded breakthrough data from {breakthrough_curve_path}")
    except:
        print("No breakthrough data available")
        bv = hardness = None
else:
    print("No breakthrough data available for plotting")
    bv = hardness = None

In [ ]:
# Create breakthrough curve plot
if bv is not None and len(bv) > 0:
    if USE_PLOTLY:
        # Interactive Plotly plot
        fig = go.Figure()
        
        # Add hardness trace
        fig.add_trace(go.Scatter(
            x=bv[:len(hardness)],
            y=hardness,
            mode='lines',
            name='Total Hardness',
            line=dict(color='blue', width=2)
        ))
        
        # Add target line
        fig.add_hline(
            y=target_hardness,
            line_dash="dash",
            line_color="red",
            annotation_text=f"Target: {target_hardness} mg/L"
        )
        
        # Add breakthrough point
        fig.add_vline(
            x=breakthrough_bv,
            line_dash="dash",
            line_color="green",
            annotation_text=f"Breakthrough: {breakthrough_bv:.1f} BV"
        )
        
        fig.update_layout(
            title='Ion Exchange Breakthrough Curve',
            xaxis_title='Bed Volumes',
            yaxis_title='Hardness (mg/L as CaCO₃)',
            height=500,
            hovermode='x unified'
        )
        
        fig.show()
    else:
        # Static matplotlib plot
        plt.figure(figsize=(10, 6))
        plt.plot(bv[:len(hardness)], hardness, 'b-', linewidth=2, label='Total Hardness')
        plt.axhline(y=target_hardness, color='r', linestyle='--', label=f'Target: {target_hardness} mg/L')
        plt.axvline(x=breakthrough_bv, color='g', linestyle='--', label=f'Breakthrough: {breakthrough_bv:.1f} BV')
        
        plt.xlabel('Bed Volumes')
        plt.ylabel('Hardness (mg/L as CaCO₃)')
        plt.title('Ion Exchange Breakthrough Curve')
        plt.grid(True, alpha=0.3)
        plt.legend()
        plt.xlim(0, min(300, max(bv) if bv else 300))
        plt.ylim(0, max(100, max(hardness)*1.1 if hardness else 100))
        plt.show()
else:
    print("⚠️ No breakthrough curve data available for visualization")

In [ ]:
# Performance metrics summary
# Get bed volume from vessel configuration
if 'vessel' in design:
    vessel_params = design['vessel']
    bed_volume_m3 = vessel_params.get('bed_volume_m3')
    if not bed_volume_m3:
        # Calculate from dimensions if available
        diameter = vessel_params.get('diameter_m', 2.0)
        bed_depth = vessel_params.get('bed_depth_m', 2.0)
        from math import pi
        bed_volume_m3 = pi * (diameter/2)**2 * bed_depth
else:
    # Use default values
    from math import pi
    bed_volume_m3 = pi * (2.0/2)**2 * 2.0

service_bv = performance.get('service_bv_to_target', 0)
treated_volume_m3 = service_bv * bed_volume_m3

performance_summary = [
    ['Service to Breakthrough', f"{service_bv:.1f}", 'BV'],
    ['Service Run Time', f"{performance.get('service_hours', 0):.1f}", 'hours'],
    ['Treated Water Volume', f"{treated_volume_m3:.0f}", 'm³'],
    ['Capacity Utilization', f"{performance.get('capacity_utilization_percent', 0):.1f}", '%'],
    ['Effluent Quality', f"{performance.get('effluent_hardness_mg_l_caco3', 0):.1f}", 'mg/L as CaCO₃']
]

df_performance = pd.DataFrame(performance_summary, columns=['Metric', 'Value', 'Units'])
display(df_performance.style.set_caption('Service Cycle Performance'))